## TensorFlow 1.x to PyTorch conversion

This is an experimental notebook to convert CHAP models from Tensorflow 1.x to PyTorch.
The notebooks is tested with the below package versions.

```
python 3.7.10
tensorflow 1.15
torch 1.13.1
onnx 1.14.1
tf2onnx 1.16.1
onnx2pytorch 0.4.1
```


In [1]:
import os
import sys
sys.path.append("../")
import logging

import tensorflow
if int(tensorflow.__version__.split(".")[0]) >= 2:
    import tensorflow.compat.v1 as tf
else:
    import tensorflow as tf

import multiprocessing

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
import pickle

from model import CNNBiLSTMModel
import torch
import numpy as np
import onnx
from onnx2pytorch import ConvertModel

/home/animeshtuf/anaconda3/envs/deep_postures/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Tensorflow weights and save them as pickle

In [2]:
tf_weights_path = "../pre-trained-models/CHAP_ALL_ADULTS"
tf_weights_pickle_path = "./pretrained_model_weights/CHAP_ALL_ADULTS.pickle"

tf.reset_default_graph()
p = max(1, multiprocessing.cpu_count()//2)
sess = tf.Session(config=tf.ConfigProto(inter_op_parallelism_threads=p, intra_op_parallelism_threads=p))
tf.saved_model.loader.load(sess, ["serve"], tf_weights_path)

vars = sess.graph.get_collection('trainable_variables')
weights = {}
for v in vars:
    weights[v.name] = sess.run(v)  # retrieve the value from the tf backend
weights_list = [(k, v) for k, v in weights.items()]
with open(tf_weights_pickle_path, 'wb') as handle:
    pickle.dump(weights_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


2024-07-22 22:15:51.830831: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2024-07-22 22:15:51.834631: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2918400000 Hz
2024-07-22 22:15:51.835006: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5c87b95223b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-07-22 22:15:51.835019: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-07-22 22:15:51.835145: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-07-22 22:15:51.835152: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-22 22:15:51.835160

INFO:tensorflow:Restoring parameters from ../pre-trained-models/CHAP_ALL_ADULTS/variables/variables


INFO:tensorflow:Restoring parameters from ../pre-trained-models/CHAP_ALL_ADULTS/variables/variables


## Convert existing Tensorflow model to ONNX model

`tf2onnx` also supports Python based API however the command line produces the most consistent results

In [3]:
!python -m tf2onnx.convert --saved-model ../pre-trained-models/CHAP_ALL_ADULTS --output ./onnx_models/CHAP_ALL_ADULTS.onnx

/home/animeshtuf/anaconda3/envs/deep_postures/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))

2024-07-22 22:15:53,020 - WARNING - From /home/animeshtuf/anaconda3/envs/deep_postures/lib/python3.7/site-packages/tf2onnx/verbose_logging.py:76: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

2024-07-22 22:15:53.021138: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-22 22:15:53,026 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-07-22 22:15:53,026 - WARNING - '--signature_def' not provided. Using all signatures.
2024-07-22 22:15:53,609 - INFO - Using tensorflow=1.15.0, onnx=1.14.1, tf2onnx=1.16.1/15c810
2024-07-22 22:15:53,610 - INFO - Using opset <onnx, 15>
2024-07-22 

# Converting to PyTorch weights

We will be manually loading the weights into the PyTorch layers. For the LSTM layers we will be using the model weights from the ONNX converted model and for all the other layers we will be using the weights from the pickle file we created earlier. We are not using just the ONNX converted model as it creates complex layers for the convolution layer and also has a limitation of having a batch size 1.

In [4]:
onnx_model_path = "./onnx_models/CHAP_ALL_ADULTS.onnx"

# Open the pickle file for reading in binary mode
with open(tf_weights_pickle_path, 'rb') as f:
    # Load the data from the pickle file
    weights_list = pickle.load(f)

# Model Configurations
amp_factor =2
bi_lstm_window_size = 42
num_classes = 2
model = CNNBiLSTMModel(amp_factor=amp_factor, bi_lstm_win_size=bi_lstm_window_size, num_classes=2, load_pretrained = True)

# Copy layers other than LSTM
with torch.no_grad():
      pt_model_params = list(model.named_parameters())
      n_tf = 0
      n_pt = 0
      while n_tf<len(weights_list):
            pt_name, pt_param = pt_model_params[n_pt]
            tf_name, tf_param = weights_list[n_tf]
            if "lstm" in tf_name:
                  n_pt=n_pt+4
                  n_tf=n_tf+2
                  continue
            # conv weights are in order NHWC for TF and NCHW for PyTorch
            if "conv" in tf_name and len(tf_param.shape) == 4:
                  tf_param = np.transpose(tf_param, (3, 2, 0, 1)).copy()  
            # dense weights are in order KN for TF and NK for PyTorch
            elif "dense" in tf_name and len(tf_param.shape) == 2:
                  tf_param = np.transpose(tf_param).copy()
            n_tf+=1
            n_pt+=1
                  
            if not tf_param.shape == pt_param.detach().numpy().shape:
                  print("Shape error", "TF", tf_name, tf_param.shape, "\tPT", pt_name, pt_param.detach().numpy().shape)
            print(f"Copying layer {tf_name} to {pt_name}")      
            pt_param.copy_(torch.tensor(tf_param, requires_grad=True, dtype=pt_param.dtype))

# Load the ONNX model
onnx_model = onnx.load(onnx_model_path)
pytorch_onnx_model = ConvertModel(onnx_model)

# Get the model's state_dict
model_weights = pytorch_onnx_model.state_dict()
lstm_weights = {}
for k in model_weights:
    if "lstm" in k.lower() and 'LSTMCellZeroState' not in k:
        lstm_weights[k] = model_weights[k]

lstm_tuple_list = [(k,v) for k,v in lstm_weights.items()]

with torch.no_grad():
      pt_model_params = list(model.named_parameters())
      n_onx = 0
      n_pt = 0
      while n_pt<len(pt_model_params) and n_onx<len(lstm_tuple_list):
            pt_name, pt_param = pt_model_params[n_pt]
            onx_name, onx_param = lstm_tuple_list[n_onx]
            if 'lstm' not in pt_name.lower():
                  n_pt+=1
                  continue
                  
            if not onx_param.shape == pt_param.detach().numpy().shape:
                  print("Shape error", "ONX", onx_param, onx_param.shape, "\tPT", pt_name, pt_param.detach().numpy().shape)
            print(f"Copying layer {onx_name} to {pt_name}")
            pt_param.copy_(torch.tensor(onx_param, requires_grad=True, dtype=pt_param.dtype))
            n_pt+=1
            n_onx+=1

Copying layer model/conv2d/kernel:0 to cnn_model.conv1.weight
Copying layer model/conv2d/bias:0 to cnn_model.conv1.bias
Copying layer model/conv2d_1/kernel:0 to cnn_model.conv2.weight
Copying layer model/conv2d_1/bias:0 to cnn_model.conv2.bias
Copying layer model/conv2d_2/kernel:0 to cnn_model.conv3.weight
Copying layer model/conv2d_2/bias:0 to cnn_model.conv3.bias
Copying layer model/conv2d_3/kernel:0 to cnn_model.conv4.weight
Copying layer model/conv2d_3/bias:0 to cnn_model.conv4.bias
Copying layer model/conv2d_4/kernel:0 to cnn_model.conv5.weight
Copying layer model/conv2d_4/bias:0 to cnn_model.conv5.bias
Copying layer model/dense/kernel:0 to cnn_model.fc.weight
Copying layer model/dense/bias:0 to cnn_model.fc.bias
Copying layer dense/kernel:0 to fc_bilstm.weight
Copying layer dense/bias:0 to fc_bilstm.bias
Automatic inference of operator: round
Copying layer LSTM_LSTM__84:0.lstm.weight_ih_l0 to bil_lstm.weight_ih_l0
Copying layer LSTM_LSTM__84:0.lstm.weight_hh_l0 to bil_lstm.weight

/home/animeshtuf/anaconda3/envs/deep_postures/lib/python3.7/site-packages/onnx2pytorch/convert/layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))
/home/animeshtuf/anaconda3/envs/deep_postures/lib/python3.7/site-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


## Save PyTorch Model

In [5]:
pytorch_model_save_path = "../pre-trained-models-pt/"
torch.save(model.state_dict(), os.path.join(pytorch_model_save_path, "CHAP_ALL_ADULTS.pth"))